In [2]:
#이미지 두개 리사이즈 후 합치기
from PIL import Image
 
image1 = Image.open('C:\\Users\\Hyeongkyu Park\\Documents\\내 Tableau 리포지토리\\이미지\\합침.jpg')
image1.show()
 
image2 = Image.open('C:\\Users\\Hyeongkyu Park\\Documents\\내 Tableau 리포지토리\\이미지\\드레이크.jpg')
image2.show()


image1 = image1.resize((600, 600))
image2 = image2.resize((300, 300))
image1_size = image1.size
image2_size = image2.size
new_image = Image.new('RGB',(2*image1_size[0], image1_size[1]), (250,250,250))
new_image.paste(image1,(0,0))
new_image.paste(image2,(image1_size[0],0))
new_image.save("C:\\Users\\Hyeongkyu Park\\Documents\\내 Tableau 리포지토리\\이미지\\합침.jpg","JPEG")
new_image.show()


In [4]:
import pandas as pd

df1 = pd.read_csv('C:\\Users\\Hyeongkyu Park\\Desktop\\데이터분석\\Datasets\\mbti\\mbti조인완성.csv', encoding='cp949')

df2 = pd.read_csv('C:\\Users\\Hyeongkyu Park\\Desktop\\데이터분석\\Datasets\\mbti\\mbti상세설명.csv', encoding='cp949')

df_join = pd.merge(df1, df2, left_on='성격유형', right_on='성격유형', how='inner')

df_join.to_csv('C:\\Users\\Hyeongkyu Park\\Desktop\\데이터분석\\Datasets\\mbti\\mbti최종.csv', encoding='cp949')

In [2]:
import os
import tkinter.ttk as ttk 
import tkinter.messagebox as msgbox 
from tkinter import *
from tkinter import filedialog  
from PIL import Image

root = Tk()
root.title("nado gui")

file_frame = Frame(root)
file_frame.pack(fill="x" , padx=5, pady=5) 

def add_file():
    files = filedialog.askopenfilenames(title="이미지 파일을 선택하세요.", \
        filetypes=(("PNG파일","*.png"),("모든파일","*")),  \
        initialdir=r"E:\PythonWorkspace\pygame_project\images")
        

    for file in files:
        list_file.insert(END,file)

def del_file():

    for index in reversed(list_file.curselection()):
        list_file.delete(index)   
        
        

def browse_dest_path():
    folder_selected = filedialog.askdirectory()
    if folder_selected == '': 
        return
    
    txt_dest_path.delete(0,END)
    txt_dest_path.insert(0, folder_selected)

def merge_image():
   
    try:
        img_width = cmb_width.get() 
        if img_width == "원본유지":
            img_width= -1 
        else:
            img_width= int(img_width)

    
        img_space = cmb_space.get()
        if img_space == "좁게" :
            img_space = 30
        elif img_space == "보통" :
            img_space = 60
        elif img_space == "넓게" :
            img_space = 90
        else :
            img_space = 0

        
        img_format = cmb_format.get().lower() 

        images = [Image.open(x) for x in list_file.get(0,END)]

        
        image_sizes = [] 
        if img_width > -1 :
            image_sizes = [(int(img_width), int(img_width * x.size[1] / x.size[0])) for x in images]
        else :
            image_sizes = [(x.size[0],x.size[1]) for x in images] 

    

        widths,heights = zip(*(image_sizes))

        max_width,total_height = max(widths),sum(heights)

        #스케치북 준비 
        # 옵션 중 스페이스 주기 위한 작업
        if img_space > 0 : 
            total_height += (img_space * (len(images) - 1))
    

        result_img = Image.new("RGB", (max_width, total_height), (255,255,255)) 
        y_offset = 0 

        
        for idx , img in enumerate(images):
            #width가 원본유지가 아닐 때에는 이미지 크기 조정
            if img_width > -1:
                img = img.resize(image_sizes[idx])

            result_img.paste(img,(0,y_offset)) 
            y_offset += (img.size[1] + img_space) #heigt 값 + 사용자가 지정한 간격

            progress = (idx + 1) / len(images) * 100 
            p_var.set(progress)
            progress_bar.update()

        # 포맷 옵션 처리
        file_name = "nado_photo." + img_format
        dest_path = os.path.join(txt_dest_path.get(), file_name) 
        result_img.save(dest_path)    
        msgbox.showinfo("알림", "작업이 완료 되었습니다.")

    except Exception as err :  # 예외처리
        msgbox.showerror("에러" , err)

#시작
def start():

    #파일 목록 확인
    if list_file.size() == 0 :   #파일이 하나도 없다면
        msgbox.showwarning("경고","이미지 파일을 추가하세요")
        return # 빠져나간다.

    #저장 경로 확인
    if len(txt_dest_path.get()) == 0 :   #파일이 하나도 없다면
        msgbox.showwarning("경고","저장 경로를 선택하세요.")
        return # 빠져나간다.

     # 이미지 통합 작업
    merge_image()   

btn_add_file = Button(file_frame, padx=5, pady=5, width=12, text="파일추가", command=add_file)  
btn_add_file.pack(side="left")

btn_del_file = Button(file_frame, padx=5, pady=5, width=12, text="선택삭제" ,command=del_file)  
btn_del_file.pack(side="right")

list_frame = Frame(root)
list_frame.pack(fill="both" , padx=5, pady=5) 

scrollbar = Scrollbar(list_frame)
scrollbar.pack(side="right" , fill="y") 

list_file = Listbox(list_frame, selectmode = "extended" , height=15 , yscrollcommand=scrollbar.set) 
list_file.pack(side="left" , fill="both" , expand=True)
scrollbar.config(command=list_file.yview) 

path_frame = LabelFrame(root, text="저장경로")
path_frame.pack(fill="x" , padx=5, pady=5 , ipady= 5) 

txt_dest_path = Entry(path_frame)
txt_dest_path.pack(side="left", fill="x", expand=True ,padx=5, pady=5 , ipady= 4) 

btn_dest_path = Button(path_frame, text="찾아보기" , width=10 , command=browse_dest_path)
btn_dest_path.pack(side="right" , padx=5, pady=5)

option_frame = LabelFrame(root, text="옵션")
option_frame.pack(padx=5, pady=5 ,ipadx=5, ipady= 10) 

lbl_width = Label(option_frame, text="가로넓이" , width=8)
lbl_width.pack(side="left")

opt_width = ["원본유지","1024","800","640"]
cmb_width = ttk.Combobox(option_frame, state="readonly" , values=opt_width , width=10)
cmb_width.current(0)
cmb_width.pack(side="left")

lbl_space = Label(option_frame, text="간격" , width=8)
lbl_space.pack(side="left")

opt_space = ["없음","좁게","보통","넓게"]
cmb_space= ttk.Combobox(option_frame, state="readonly" , values=opt_space , width=10)
cmb_space.current(0)
cmb_space.pack(side="left")
 
lbl_format = Label(option_frame, text="포맷" , width=8)
lbl_format.pack(side="left")

opt_format = ["PNG","JPG","BMP"]
cmb_format= ttk.Combobox(option_frame, state="readonly" , values=opt_format , width=10)
cmb_format.current(0)
cmb_format.pack(side="left")

progress_frame = LabelFrame(root, text="진행상황")
progress_frame.pack(fill="x" , padx=5, pady=5 , ipady= 5) 

p_var = DoubleVar() 
progress_bar = ttk.Progressbar(progress_frame, maximum = 100 , variable=p_var) 
progress_bar.pack(fill="x" ,padx=5, pady=5)

run_frame = Frame(root)
run_frame.pack(fill="x"  , padx=5, pady=5)

btn_close = Button(run_frame, padx=5, pady=5, width=12, text="닫기" , command=root.quit) 
btn_close.pack(side="right" , padx=5, pady=5)

btn_start = Button(run_frame, padx=5, pady=5, width=12, text="시작" , command=start)  # command=start 추가
btn_start.pack(side="right" , padx=5, pady=5)

root.resizable(False, False)
root.mainloop() 
